In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
import os 
os.chdir('drive/My Drive/CS4225Test')

In [4]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='iso-8859-1', header = None)

df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
sid = SentimentIntensityAnalyzer()
def sentiment_analysis(comments):
    ss = sid.polarity_scores(comments)
    return ss['compound']

In [6]:
df_small = df[0:100000].copy()

import time
start = time.process_time()

df_small['sentiment_score'] = df_small[5].apply(sentiment_analysis)
  
print(time.process_time() - start)


17.216184436


In [7]:
df_small[5000:5050]

,0,1,2,3,4,5,sentiment_score
5000,0,1468963818,Tue Apr 07 04:36:55 PDT 2009,NO_QUERY,Nezbo,"@LouLouK Gisburn is great, there are some nice...",0.7845
5001,0,1468964164,Tue Apr 07 04:37:00 PDT 2009,NO_QUERY,amandathomass,Orange juice and toothpaste mouth combo =,0.0000
5002,0,1468964301,Tue Apr 07 04:37:03 PDT 2009,NO_QUERY,wenuitun,And hating that watching House will kill me! I...,-0.8595
5003,0,1468965099,Tue Apr 07 04:37:16 PDT 2009,NO_QUERY,CaitlinMcKeon,"Sitting in my car, waiting for my mum to come ...",0.0000
5004,0,1468965277,Tue Apr 07 04:37:19 PDT 2009,NO_QUERY,slipandstumble,@zampeachie @Farscale To answer both of you: I...,0.0258
5005,0,1468965461,Tue Apr 07 04:37:21 PDT 2009,NO_QUERY,adriankhall,Is home! Been @ WCH since 8:30AM this morning....,-0.5090
5006,0,1468965663,Tue Apr 07 04:37:26 PDT 2009,NO_QUERY,MarinaMina,needs some tlc,0.0000
5007,0,1468965696,Tue Apr 07 04:37:26 PDT 2009,NO_QUERY,russbaker,@jemcam hey i've finished homework but B is ti...,0.0000
5008,0,1468965947,Tue Apr 07 04:37:30 PDT 2009,NO_QUERY,ThiaHeartsNick,@cheechbud Aww thats beautiful in a sad way bu...,0.1027
5009,0,1468965968,Tue Apr 07 04:37:31 PDT 2009,NO_QUERY,Maddie_14,Riddle Answer: Romeo and Juliet are fish!!! An...,-0.6093


# TIANHANG DATA


In [18]:
df_new = pd.read_csv('part_00000_tid_8026940221144187376_48818ee7_6db7_4c51_af40_3924fa515d8d.csv', header = None)
df_new.head()

,0,1,2,3,4
0,2020-02-10,en,Philippines,repatriated filipinos quarantine tested symptoms day pm coronavirus ncov |,"0.01, 0.94, 0.05"
1,2020-02-10,en,United States,donated n masks amp; crucial supplies going left outside waste rainfalls wuhanvirus…,"0.01, 0.02, 0.97"
2,2020-02-10,en,Hong Kong,wrote column support taiwan's participation global response coronavirus membership taiwan i…,"0.1, 0.85, 0.05"
3,2020-02-10,en,Malaysia,protect real life viruses viruses,"0.12, 0.29, 0.59"
4,2020-02-10,en,United States,incredible body work good friend amazing work it's eye opener respect to…,"1, 0, 0"


# compare tianhang data


In [9]:
def get_posneuneg (theString):
  theList = theString.split(',')
  newList = []
  for i in theList:
    newList.append(np.float(i))
  return newList.index(max(newList)) + 1


In [19]:
df_new_score = df_new.copy()
pd.set_option('display.max_colwidth', 100000)


df_new_score['score'] = df_new_score[3].apply(sentiment_analysis)

df_new_score.head()

,0,1,2,3,4,score
0,2020-02-10,en,Philippines,repatriated filipinos quarantine tested symptoms day pm coronavirus ncov |,"0.01, 0.94, 0.05",0.0000
1,2020-02-10,en,United States,donated n masks amp; crucial supplies going left outside waste rainfalls wuhanvirus…,"0.01, 0.02, 0.97",-0.4215
2,2020-02-10,en,Hong Kong,wrote column support taiwan's participation global response coronavirus membership taiwan i…,"0.1, 0.85, 0.05",0.4019
3,2020-02-10,en,Malaysia,protect real life viruses viruses,"0.12, 0.29, 0.59",0.3818
4,2020-02-10,en,United States,incredible body work good friend amazing work it's eye opener respect to…,"1, 0, 0",0.9186


In [20]:
df_new_score['AzureSentiment'] = df_new_score[4].apply(get_posneuneg)

def check_sentiment (sentiment, threshold):
  if sentiment > threshold:
    return 1
  elif sentiment < (-1 * threshold):
    return 3
  else:
    return 2

wrong_entries = []
threshold_used = []

threshold = 0
df_new_score['NLTK'] = np.vectorize(check_sentiment)(df_new_score['score'], threshold)
df_new_score_different = df_new_score[df_new_score['AzureSentiment'] != df_new_score['NLTK']]
wrong_entries.append(df_new_score_different.shape[0])
threshold_used.append(threshold)

# min(wrong_entries)
print(min(wrong_entries))
print(df_new_score.shape[0])
threshold_used[wrong_entries.index(min(wrong_entries))]

592
1422


0

In [21]:
df_new_score.columns = ['Date', 'Language', 'Country', 'Tweet', 'AzureRaw', 'NLTK Raw', 'Azure Sentiment', 'NLTK Sentiment']

# df_new_score.to_csv('CompareNLTKWithAzureSentiment.csv')
df_new_score.head(50)

,Date,Language,Country,Tweet,AzureRaw,NLTK Raw,Azure Sentiment,NLTK Sentiment
0,2020-02-10,en,Philippines,repatriated filipinos quarantine tested symptoms day pm coronavirus ncov |,"0.01, 0.94, 0.05",0.0000,2,2
1,2020-02-10,en,United States,donated n masks amp; crucial supplies going left outside waste rainfalls wuhanvirus…,"0.01, 0.02, 0.97",-0.4215,3,3
2,2020-02-10,en,Hong Kong,wrote column support taiwan's participation global response coronavirus membership taiwan i…,"0.1, 0.85, 0.05",0.4019,2,1
3,2020-02-10,en,Malaysia,protect real life viruses viruses,"0.12, 0.29, 0.59",0.3818,3,1
4,2020-02-10,en,United States,incredible body work good friend amazing work it's eye opener respect to…,"1, 0, 0",0.9186,1,1
5,2020-02-10,en,United States,uk man linked seven coronavirus infections countries,"0.01, 0.97, 0.02",0.0000,2,2
6,2020-02-10,en,Canada,spraying,"0.08, 0.85, 0.07",0.0000,2,2
7,2020-02-10,en,China,scientists various pharmaceutical companies institutions tried create vaccine stop spread o…,"0.01, 0.93, 0.06",-0.0258,2,3
8,2020-02-10,en,Hong Kong,local officials remote corners—from iceland antarctica—are rushing prepare possible arrival virus reflection china’s global reach quickly fear spreading,"0, 0.04, 0.96",-0.4767,3,3
9,2020-02-10,en,Malaysia,imask active coronavirus cancellation starting rmk,"0.01, 0.97, 0.02",0.4019,2,1
